In [19]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import csv
import os

In [106]:
# creating function to get all the onformation from all url
#define fun
def get_prop_info (url):

    # Create request to access url 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")

    # get the Zip code fron url immo web url with regex
    zip_pattern = re.findall("([/]+[\d{4}]+[/])",str(url))
    zip_code = zip_pattern[0].replace("/","")

    # Get price of the property
    price = soup.find("p", class_ = "classified__price").text
    price = price.split()[0]

    # get zip and price in one dictionary
    zip_price_key = ["zip", "price"]
    zip_price_values = [zip_code, price]
    dict_zip_price = dict(zip(zip_price_key, zip_price_values))
    
    #get all the key and their respective values from all table and save it in dictionary 
    container = soup.find_all("tr")
    container_dict = {}
    container_keys_list = []
    container_values_list = []

    for row in container:
        for th in row.find_all("th"):
            container_keys_list.append(th.contents[0].strip())

            for td in row.find_all("td"):
                container_values_list.append(td.contents[0].strip())

    container_dict = dict(zip(container_keys_list, container_values_list))

    # save all the data in dictionary
    final_prop_dict = {**dict_zip_price,**container_dict}
    return final_prop_dict

In [86]:
# create instance of WebDriver
driver = webdriver.Firefox()

links_list = []

# open file in write mode
file_name = open(r"all_links.txt", "w")

for page in range(1, 333):
    l = f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&page={page}&orderBy=relevance"

    driver.get(l)
    elements = driver.find_elements(by=By.XPATH, value="//a[@class='card__title-link']")

    for elem in elements:
        href = elem.get_attribute("href")
        links_list.append(href)
        
#close the driver
driver.close()

#store all links from list to txt file
with open(r"all_links.txt", "w") as fp:
    for item in links_list:    
        fp.write('\n'.join(links_list))

In [110]:
# open each url from all links file

proprties = []

with open(r"all_links.txt", "r")as f:
    for link in f.readlines()[:10000]:

        info = get_prop_info(link)
        proprties.append(info)

df = pd.DataFrame(proprties)
df.to_csv("immo_data.csv")
